In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('https://drive.google.com/uc?id=1OYMa1QBf6Lt2vVABrFJi8FO4UFhieVRv')
customer = pd.read_csv('https://drive.google.com/uc?id=1uyHiU-iKwJi8ht3TqBqmmFojvSc0CY_N')
transactions = pd.read_csv('transactions_hm.csv')

In [ ]:
# 각 테이블에 대한 열과 행 개수를 표현해주세요.
articles.shape # (105542, 25)
customer.shape # (1048575, 6)
transactions.shape # (1048575, 5)

In [ ]:
# 각 테이블에 대한 컬럼타입과 통계량을 보여주세요.
articles.info()
articles.describe()
customer.info()
customer.describe()
transactions.info()
transactions.describe()

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.
df_total_table = pd.merge(transactions, customer, on='customer_id', how='inner')
df_total_table = pd.merge(df_total_table, articles, on='article_id', how='inner')
df_total_table.head()

In [ ]:
# NULL 값이 존재하는 경우, 대체 or 제거 모두 가능합니다.
articles.isnull().sum() # 제품 상세 설명 null 이므로 대체/제거 하지 않음
customer.isnull().sum() # 뉴스 레터 주기 null 값이나 큰 영향을 주지 않아 대체/제거 하지 않음.
transactions.isnull().sum() # null 값 없음

In [ ]:
df_total_table["t_dat"].head()

In [ ]:
df_total_table["t_datetime"] = pd.to_datetime(df_total_table["t_dat"])
df_total_table["month"] = df_total_table["t_datetime"].dt.month

# 실제 금액처럼 보이게 price 정규화 해제 (예: 100,000 곱하기)
# df_total_table["price_real"] = df_total_table["price"] * 100000

# 월 별 판매량과 주문 수량 집계
df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 월 별 통계량
# df_total_table.groupby("month")["price"].agg(["mean", "median", "max", "min", "count"])

In [ ]:
import matplotlib.pyplot as plt

# 월별 집계
monthly_summary = df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 통계값 계산
mean_sales = monthly_summary['total_sales'].mean()
median_sales = monthly_summary['total_sales'].median()
mean_orders = monthly_summary['order_count'].mean()
median_orders = monthly_summary['order_count'].median()

# 시각화 시작
fig, ax1 = plt.subplots(figsize=(12, 6))

# ① 실선: 판매 매출
ax1.plot(
    monthly_summary.index,
    monthly_summary['total_sales'],
    color='tab:blue',
    marker='o',
    linestyle='-',
    label='Total Sales'
)
ax1.axhline(mean_sales, color='tab:blue', linestyle=':', label='Sales Mean')

# ✅ 매출 데이터 라벨 (왼쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['total_sales']):
    ax1.text(x - 0.15, y, f'{y:,.0f}', color='tab:blue', fontsize=9, ha='right', va='bottom')

ax1.set_xlabel('Month')
ax1.set_ylabel('Total Sales (₩)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_xticks(monthly_summary.index)

# ② 실선: 주문량
ax2 = ax1.twinx()
ax2.plot(
    monthly_summary.index,
    monthly_summary['order_count'],
    color='tab:red',
    marker='s',
    linestyle='-',
    label='Order Count'
)
ax2.axhline(mean_orders, color='tab:red', linestyle=':', label='Order Mean')

# ✅ 주문 수 데이터 라벨 (오른쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['order_count']):
    ax2.text(x + 0.15, y, f'{int(y)}', color='tab:red', fontsize=9, ha='left', va='bottom')

ax2.set_ylabel('Order Count', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# 제목
plt.title('월별 판매 매출 & 주문량 (평균선/중앙값 포함 + 좌우 분리 라벨)', fontsize=14)
fig.tight_layout()

# 범례 병합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
fig.legend(lines1 + lines2, labels1 + labels2, loc='upper center', ncol=3)

plt.show()
